In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
try:
    imp = pd.read_csv("results.tx", header=None).values
    pos = np.where(imp[:, -1] == "O")[0]+1
    pos = np.insert(pos, 0, 0)
    his = imp[:, 1] - imp[:, 0]
    los = imp[:, 2] - imp[:, 0]
    cls = imp[:, 3] - imp[:, 0]
    tb = np.zeros((len(pos) , 4))
    for i in range(0, len(pos)-1):
        if i > 0:
            tb[i, 0] = tb[i-1, 3]
        tb[i, 1] = tb[i, 0] + 100 * np.sum(his[pos[i] :pos[i+1]])
        tb[i, 2] = tb[i, 0] + 100 * np.sum(los[pos[i] :pos[i+1]])
        tb[i, 3] = tb[i, 0] + 100 * np.sum(cls[pos[i] :pos[i+1]])
    tb_df = pd.DataFrame(tb, columns=['Open', 'High', 'Low', 'Close'])
    res = tb_df['Close'] - tb_df['Open']
    win = res[res > 0]
    los = res[res < 0]
    num_trades = len(res)
    win_prob = len(win) / num_trades if num_trades > 0 else 0
    los_prob = len(los) / num_trades if num_trades > 0 else 0
    win_mean = win.mean() if len(win) > 0 else 0
    los_mean = los.mean() if len(los) > 0 else 0
    win_loss_ratio = -win_mean / los_mean if los_mean != 0 else float('inf')
    expectation = win_mean * win_prob + los_mean*los_prob
    fig, ax = plt.subplots(figsize=(10, 5))
    candlestick_ohlc(ax, tb_df.reset_index().values, width=0.6, colorup='green', colordown='red')
    ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True, prune='both'))
    ax.set_xlabel('Trade Count')
    ax.set_ylabel('Profit and Loss')
    ax.set_title('Performance Chart')
    sign='+'
    if expectation<0: sign='-'
    stats_text = (
        f"Number of trades:     {num_trades}\n"
        f"Win probability:         {100*win_prob:.2f}%\n"
        f"Loss probability:        {100*los_prob:.2f}%\n"
        f"Average Win:        +${win_mean:.2f}\n"
        f"Average Loss:        - ${-los_mean:.2f}\n"
        f"Win/Loss ratio:           {win_loss_ratio:.2f}\n"
        f"Expect per trade:   {sign}${np.abs(expectation):.2f}"
    )
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(1.02, 1.00, stats_text, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=props)
    plt.show()
except:
    print("Error loading results file.")